In [1]:
import os
import json
import string
import random
inp = "train"

In [2]:
AUDIO_FOLDER = os.getcwd()+r"/audio/"+inp+r"/"
DATA_FOLDER = os.getcwd()+r"/data/"+inp+r"/"

In [15]:
# Renaming files
listAudioFiles = os.listdir(AUDIO_FOLDER)
jsonFileRenames = DATA_FOLDER+"file_renames.json"

if os.path.isfile(jsonFileRenames):
    with open(jsonFileRenames, mode='r', encoding='utf-8-sig') as f:
        dictFileRenames = json.load(f)
else:
    new_names = list()
    while len(new_names)<len(listAudioFiles):
        new_names = list(set([(''.join(random.choices(string.ascii_lowercase, k=8)))+".wav" for _ in range(len(listAudioFiles))]))
    dictFileRenames = dict(zip(new_names, listAudioFiles))
    with open(jsonFileRenames, mode='w', encoding='utf-8-sig') as f:
        json.dump(dictFileRenames, f)

for new_name, old_name in dictFileRenames.items():
    if old_name in listAudioFiles:
        os.rename(AUDIO_FOLDER+old_name, AUDIO_FOLDER+new_name)

In [20]:
# Replacing filenames in all files
transcription_files = os.listdir(DATA_FOLDER+r"preTrimmed/")
print("Trimming dataset based on available audio files...")
for filename in transcription_files:
    data = list()
    
    with open(DATA_FOLDER+r"/preTrimmed/"+filename, mode='r', encoding='utf-8-sig') as f:
        for line in f.read().split("\n"):
            if (line[:21]+".wav") in dictFileRenames.values():
                new_file_name = list(dictFileRenames.keys())[list(dictFileRenames.values()).index(line[:21]+".wav")]
                new_line = line.replace(line[:21], new_file_name[:-4])
                data.append(new_line)

    with open(DATA_FOLDER+filename, mode='w', encoding='utf-8-sig') as f:
        for line in sorted(data):
            f.write(line+"\n")
    
    print("DONE: "+filename)

Trimming dataset based on available audio files...
DONE: segments
DONE: spk2utt
DONE: text
DONE: utt2spk


In [22]:
with open(DATA_FOLDER+"wav.scp", mode='w', encoding='utf-8-sig') as f:
    for filename in sorted(list(dictFileRenames.keys())):
        f.write(filename[:-4]+"\t"+AUDIO_FOLDER+filename+"\n")

print("DONE: wav.scp")

DONE: wav.scp


In [ ]:
# Reversing name changes
for new_name, old_name in dictFileRenames.items():
    if new_name in listAudioFiles:
        os.rename(AUDIO_FOLDER+new_name, AUDIO_FOLDER+old_name)